In [1]:
import pandas as pd
import numpy as np

In [89]:
train=pd.read_csv("Kaggle_Training_Dataset_v2.csv")
test=pd.read_csv("Kaggle_Test_Dataset_v2.csv")
train.shape,test.shape

((1687861, 23), (242076, 23))

In [3]:
train['lead_time'].skew()
train['lead_time']=train['lead_time'].fillna(train['lead_time'].median())

In [4]:
train.dropna(axis=0,inplace=True)

In [5]:
train.isnull().sum()

sku                  0
national_inv         0
lead_time            0
in_transit_qty       0
forecast_3_month     0
forecast_6_month     0
forecast_9_month     0
sales_1_month        0
sales_3_month        0
sales_6_month        0
sales_9_month        0
min_bank             0
potential_issue      0
pieces_past_due      0
perf_6_month_avg     0
perf_12_month_avg    0
local_bo_qty         0
deck_risk            0
oe_constraint        0
ppap_risk            0
stop_auto_buy        0
rev_stop             0
went_on_backorder    0
dtype: int64

In [6]:
train.drop(columns="sku",inplace=True)

In [7]:
cat_cols=[i for i in train.columns if train[i].dtype=="object" ]
cat_cols

['potential_issue',
 'deck_risk',
 'oe_constraint',
 'ppap_risk',
 'stop_auto_buy',
 'rev_stop',
 'went_on_backorder']

In [8]:
num_cols=train.columns[train.dtypes!="object"]
num_cols

Index(['national_inv', 'lead_time', 'in_transit_qty', 'forecast_3_month',
       'forecast_6_month', 'forecast_9_month', 'sales_1_month',
       'sales_3_month', 'sales_6_month', 'sales_9_month', 'min_bank',
       'pieces_past_due', 'perf_6_month_avg', 'perf_12_month_avg',
       'local_bo_qty'],
      dtype='object')

In [9]:
cor_matrix=train[num_cols].corr()
cols_corr=set()
for i in range(len(cor_matrix.columns)):
    for j in range(i):
        if cor_matrix.iloc[i,j] > 0.90:
            col_name=cor_matrix.columns[i]
            cols_corr.add(col_name)
cols_corr

{'forecast_6_month',
 'forecast_9_month',
 'perf_12_month_avg',
 'sales_3_month',
 'sales_6_month',
 'sales_9_month'}

In [12]:
drop_cols=['forecast_6_month','forecast_9_month','perf_12_month_avg','sales_3_month','sales_6_month','sales_9_month','sku']

In [13]:
drop_cols

['forecast_6_month',
 'forecast_9_month',
 'perf_12_month_avg',
 'sales_3_month',
 'sales_6_month',
 'sales_9_month',
 'sku']

In [17]:
import warnings
warnings.filterwarnings("ignore")

In [20]:
print(cat_cols)

['potential_issue', 'deck_risk', 'oe_constraint', 'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder']


In [73]:
cat_cols=['potential_issue', 'deck_risk', 'oe_constraint', 'ppap_risk', 'stop_auto_buy', 'rev_stop']

In [29]:
print(list(new.columns[new.dtypes!="object"]))

['national_inv', 'lead_time', 'in_transit_qty', 'forecast_3_month', 'sales_1_month', 'min_bank', 'pieces_past_due', 'perf_6_month_avg', 'local_bo_qty']


In [30]:
num_cols=['national_inv', 'lead_time', 'in_transit_qty', 'forecast_3_month', 'sales_1_month', 'min_bank', 'pieces_past_due', 'perf_6_month_avg', 'local_bo_qty']

In [74]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder,StandardScaler


num_pipeline=Pipeline(
steps=[
    ('imputer',SimpleImputer(strategy="median")),
    ('scaler',StandardScaler())
])

In [75]:
cat_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy="most_frequent")),
        ('labelencoder',OrdinalEncoder(categories=[potential_issue,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop]))
    ]
)


In [76]:
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,num_cols),
    ('cat_pipeline',cat_pipeline,cat_cols)
])

In [82]:
train.columns

Index(['national_inv', 'lead_time', 'in_transit_qty', 'forecast_3_month',
       'sales_1_month', 'min_bank', 'potential_issue', 'pieces_past_due',
       'perf_6_month_avg', 'local_bo_qty', 'deck_risk', 'oe_constraint',
       'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder'],
      dtype='object')

In [77]:
nes=preprocessor.fit_transform(train)

In [78]:
preprocessor.get_feature_names_out()

array(['num_pipeline__national_inv', 'num_pipeline__lead_time',
       'num_pipeline__in_transit_qty', 'num_pipeline__forecast_3_month',
       'num_pipeline__sales_1_month', 'num_pipeline__min_bank',
       'num_pipeline__pieces_past_due', 'num_pipeline__perf_6_month_avg',
       'num_pipeline__local_bo_qty', 'cat_pipeline__potential_issue',
       'cat_pipeline__deck_risk', 'cat_pipeline__oe_constraint',
       'cat_pipeline__ppap_risk', 'cat_pipeline__stop_auto_buy',
       'cat_pipeline__rev_stop'], dtype=object)

In [81]:
nes.shape

(1586967, 15)

In [59]:
train.dropna(inplace=True)

In [60]:
cat_dict={}
for i in cat_cols:
    cat_dict[i]=[i for i in train[i].unique()]
cat_dict


{'potential_issue': ['No', 'Yes'],
 'deck_risk': ['No', 'Yes'],
 'oe_constraint': ['No', 'Yes'],
 'ppap_risk': ['No', 'Yes'],
 'stop_auto_buy': ['Yes', 'No'],
 'rev_stop': ['No', 'Yes'],
 'went_on_backorder': ['No', 'Yes']}

In [63]:
[i for i in cat_dict.keys()]

['potential_issue',
 'deck_risk',
 'oe_constraint',
 'ppap_risk',
 'stop_auto_buy',
 'rev_stop',
 'went_on_backorder']

In [62]:
potential_issue=['No', 'Yes']
deck_risk=['No', 'Yes']
oe_constraint=['No', 'Yes']
ppap_risk=['No', 'Yes']
stop_auto_buy=['Yes', 'No']
rev_stop= ['No', 'Yes']
#went_on_backorder=['No', 'Yes'])]

In [100]:

def preprocess(df):
    #train['lead_time']=train['lead_time'].fillna(train['lead_time'].median())
    drop_cols=['forecast_6_month','forecast_9_month','perf_12_month_avg','sales_3_month','sales_6_month','sales_9_month','sku']
    df.drop(columns=drop_cols,inplace=True)
    cat_cols=['potential_issue', 'deck_risk', 'oe_constraint', 'ppap_risk', 'stop_auto_buy', 'rev_stop']
    num_cols=['national_inv', 'lead_time', 'in_transit_qty', 'forecast_3_month', 'sales_1_month', 'min_bank', 'pieces_past_due', 'perf_6_month_avg', 'local_bo_qty']
    num_pipeline=Pipeline(
        steps=[
            ('imputer',SimpleImputer(strategy="median")),
            ('scaler',StandardScaler())
        ])
    potential_issue=['No', 'Yes']
    deck_risk=['No', 'Yes']
    oe_constraint=['No', 'Yes']
    ppap_risk=['No', 'Yes']
    stop_auto_buy=['Yes', 'No']
    rev_stop= ['No', 'Yes']
    cat_pipeline = Pipeline(
        steps=[
            ('imputer', SimpleImputer(strategy="most_frequent")),
            ('labelencoder',OrdinalEncoder(categories=[potential_issue,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop]))
        ]
    )
    preprocessor=ColumnTransformer([
        ('num_pipeline',num_pipeline,num_cols),
        ('cat_pipeline',cat_pipeline,cat_cols)
    ])
    new_df=preprocessor.fit_transform(df)
    return pd.DataFrame(new_df,columns=df.columns)


    

In [101]:
target='went_on_backorder'

In [102]:
train.columns

Index(['sku', 'national_inv', 'lead_time', 'in_transit_qty',
       'forecast_3_month', 'forecast_6_month', 'forecast_9_month',
       'sales_1_month', 'sales_3_month', 'sales_6_month', 'sales_9_month',
       'min_bank', 'potential_issue', 'pieces_past_due', 'perf_6_month_avg',
       'perf_12_month_avg', 'local_bo_qty', 'deck_risk', 'oe_constraint',
       'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder'],
      dtype='object')

In [103]:
new_train=preprocess(train.drop(columns=target))
new_train.head()

,national_inv,lead_time,in_transit_qty,forecast_3_month,sales_1_month,min_bank,potential_issue,pieces_past_due,perf_6_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop
0,-0.016752,0.017553,-0.032808,-0.035436,-0.029004,-0.042050,-0.008659,-3.469150,-0.018577,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.016684,0.163710,-0.032808,-0.035436,-0.029004,-0.042050,-0.008659,0.296052,-0.018577,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.016684,0.017553,-0.032808,-0.035436,-0.029004,-0.042050,-0.008659,-3.469150,-0.018577,0.0,1.0,0.0,0.0,0.0,0.0
3,-0.016516,0.017553,-0.032808,-0.035436,-0.029004,-0.041253,-0.008659,0.262538,-0.018577,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.016482,0.017553,-0.032808,-0.035436,-0.029004,-0.040457,-0.008659,-3.469150,-0.018577,0.0,1.0,0.0,0.0,0.0,0.0
